In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import re
import warnings
import sklearn
import scipy
from scipy.stats import shapiro, ttest_ind, mannwhitneyu
from lifelines import CoxPHFitter
from sklearn.model_selection import KFold
import json

warnings.filterwarnings("ignore")

Fit univariate model for each feature and save p-values

In [ ]:
my_path = '~/mounts/research/husdatalake/disease/scripts/Preleukemia/oona_new'

In [ ]:
data_path = '~/mounts/research/husdatalake/disease/processed_data/Preleukemia/'

In [ ]:
disease = 'MDS'

In [ ]:
if disease == 'de_novo_AML':
    dis = 'de novo AML'
else:
    dis = disease

In [ ]:
# List of feature candidates
features = ['sukupuoli_selite',
'age',
'rows_in_last_month',
'rows_in_last_year',
'b_baso_e9_l_trend_from_1095_d',
'b_baso_e9_l_trend_from_1825_d',
'b_baso_e9_l_trend_from_365_d',
'b_baso_e9_l_tulos_norm',
'b_baso_e9_l_tulos_norm_val_1095_n',
'b_baso_e9_l_tulos_norm_val_1825_n',
'b_baso_e9_l_tulos_norm_val_365_n',
'b_eos_e9_l_trend_from_1095_d',
'b_eos_e9_l_trend_from_1825_d',
'b_eos_e9_l_trend_from_365_d',
'b_eos_e9_l_tulos_norm',
'b_eos_e9_l_tulos_norm_val_1095_n',
'b_eos_e9_l_tulos_norm_val_1825_n',
'b_eos_e9_l_tulos_norm_val_365_n',
'b_erblast_e9_l_trend_from_1095_d',
'b_erblast_e9_l_trend_from_1825_d',
'b_erblast_e9_l_trend_from_365_d',
'b_erblast_e9_l_tulos_norm',
'b_erblast_e9_l_tulos_norm_val_1095_n',
'b_erblast_e9_l_tulos_norm_val_1825_n',
'b_erblast_e9_l_tulos_norm_val_365_n',
'b_eryt_e12_l_trend_from_1095_d',
'b_eryt_e12_l_trend_from_1825_d',
'b_eryt_e12_l_trend_from_365_d',
'b_eryt_e12_l_tulos_norm',
'b_eryt_e12_l_tulos_norm_val_1095_n',
'b_eryt_e12_l_tulos_norm_val_1825_n',
'b_eryt_e12_l_tulos_norm_val_365_n',
'b_hb_g_l_trend_from_1095_d',
'b_hb_g_l_trend_from_1825_d',
'b_hb_g_l_trend_from_365_d',
'b_hb_g_l_tulos_norm',
'b_hb_g_l_tulos_norm_val_1095_n',
'b_hb_g_l_tulos_norm_val_1825_n',
'b_hb_g_l_tulos_norm_val_365_n',
'b_hkr_percent_trend_from_1095_d',
'b_hkr_percent_trend_from_1825_d',
'b_hkr_percent_trend_from_365_d',
'b_hkr_percent_tulos_norm',
'b_hkr_percent_tulos_norm_val_1095_n',
'b_hkr_percent_tulos_norm_val_1825_n',
'b_hkr_percent_tulos_norm_val_365_n',
'b_ly_e9_l_trend_from_1095_d',
'b_ly_e9_l_trend_from_1825_d',
'b_ly_e9_l_trend_from_365_d',
'b_ly_e9_l_tulos_norm',
'b_ly_e9_l_tulos_norm_val_1095_n',
'b_ly_e9_l_tulos_norm_val_1825_n',
'b_ly_e9_l_tulos_norm_val_365_n',
'b_monos_e9_l_trend_from_1095_d',
'b_monos_e9_l_trend_from_1825_d',
'b_monos_e9_l_trend_from_365_d',
'b_monos_e9_l_tulos_norm',
'b_monos_e9_l_tulos_norm_val_1095_n',
'b_monos_e9_l_tulos_norm_val_1825_n',
'b_monos_e9_l_tulos_norm_val_365_n',
'b_neut_e9_l_trend_from_1095_d',
'b_neut_e9_l_trend_from_1825_d',
'b_neut_e9_l_trend_from_365_d',
'b_neut_e9_l_tulos_norm',
'b_neut_e9_l_tulos_norm_val_1095_n',
'b_neut_e9_l_tulos_norm_val_1825_n',
'b_neut_e9_l_tulos_norm_val_365_n',
'b_plt_e9_l_trend_from_1095_d',
'b_plt_e9_l_trend_from_1825_d',
'b_plt_e9_l_trend_from_365_d',
'b_plt_e9_l_tulos_norm',
'b_plt_e9_l_tulos_norm_val_1095_n',
'b_plt_e9_l_tulos_norm_val_1825_n',
'b_plt_e9_l_tulos_norm_val_365_n',
'b_wbc_e9_l_trend_from_1095_d',
'b_wbc_e9_l_trend_from_1825_d',
'b_wbc_e9_l_trend_from_365_d',
'b_wbc_e9_l_tulos_norm',
'b_wbc_e9_l_tulos_norm_val_1095_n',
'b_wbc_e9_l_tulos_norm_val_1825_n',
'b_wbc_e9_l_tulos_norm_val_365_n',
'e_mch_pg_trend_from_1095_d',
'e_mch_pg_trend_from_1825_d',
'e_mch_pg_trend_from_365_d',
'e_mch_pg_tulos_norm',
'e_mch_pg_tulos_norm_val_1095_n',
'e_mch_pg_tulos_norm_val_1825_n',
'e_mch_pg_tulos_norm_val_365_n',
'e_mchc_g_l_trend_from_1095_d',
'e_mchc_g_l_trend_from_1825_d',
'e_mchc_g_l_trend_from_365_d',
'e_mchc_g_l_tulos_norm',
'e_mchc_g_l_tulos_norm_val_1095_n',
'e_mchc_g_l_tulos_norm_val_1825_n',
'e_mchc_g_l_tulos_norm_val_365_n',
'e_mcv_fl_trend_from_1095_d',
'e_mcv_fl_trend_from_1825_d',
'e_mcv_fl_trend_from_365_d',
'e_mcv_fl_tulos_norm',
'e_mcv_fl_tulos_norm_val_1095_n',
'e_mcv_fl_tulos_norm_val_1825_n',
'e_mcv_fl_tulos_norm_val_365_n',
'e_rdw_percent_trend_from_1095_d',
'e_rdw_percent_trend_from_1825_d',
'e_rdw_percent_trend_from_365_d',
'e_rdw_percent_tulos_norm',
'e_rdw_percent_tulos_norm_val_1095_n',
'e_rdw_percent_tulos_norm_val_1825_n',
'e_rdw_percent_tulos_norm_val_365_n',
'e_retic_percent_trend_from_1095_d',
'e_retic_percent_trend_from_1825_d',
'e_retic_percent_trend_from_365_d',
'e_retic_percent_tulos_norm',
'e_retic_percent_tulos_norm_val_1095_n',
'e_retic_percent_tulos_norm_val_1825_n',
'e_retic_percent_tulos_norm_val_365_n',
'l_baso_percent_trend_from_1095_d',
'l_baso_percent_trend_from_1825_d',
'l_baso_percent_trend_from_365_d',
'l_baso_percent_tulos_norm',
'l_baso_percent_tulos_norm_val_1095_n',
'l_baso_percent_tulos_norm_val_1825_n',
'l_baso_percent_tulos_norm_val_365_n',
'l_blast_percent_trend_from_1095_d',
'l_blast_percent_trend_from_1825_d',
'l_blast_percent_trend_from_365_d',
'l_blast_percent_tulos_norm',
'l_blast_percent_tulos_norm_val_1095_n',
'l_blast_percent_tulos_norm_val_1825_n',
'l_blast_percent_tulos_norm_val_365_n',
'l_eos_percent_trend_from_1095_d',
'l_eos_percent_trend_from_1825_d',
'l_eos_percent_trend_from_365_d',
'l_eos_percent_tulos_norm',
'l_eos_percent_tulos_norm_val_1095_n',
'l_eos_percent_tulos_norm_val_1825_n',
'l_eos_percent_tulos_norm_val_365_n',
'l_ly_percent_trend_from_1095_d',
'l_ly_percent_trend_from_1825_d',
'l_ly_percent_trend_from_365_d',
'l_ly_percent_tulos_norm',
'l_ly_percent_tulos_norm_val_1095_n',
'l_ly_percent_tulos_norm_val_1825_n',
'l_ly_percent_tulos_norm_val_365_n',
'l_metam_percent_trend_from_1095_d',
'l_metam_percent_trend_from_1825_d',
'l_metam_percent_trend_from_365_d',
'l_metam_percent_tulos_norm',
'l_metam_percent_tulos_norm_val_1095_n',
'l_metam_percent_tulos_norm_val_1825_n',
'l_metam_percent_tulos_norm_val_365_n',
'l_mono_percent_trend_from_1095_d',
'l_mono_percent_trend_from_1825_d',
'l_mono_percent_trend_from_365_d',
'l_mono_percent_tulos_norm',
'l_mono_percent_tulos_norm_val_1095_n',
'l_mono_percent_tulos_norm_val_1825_n',
'l_mono_percent_tulos_norm_val_365_n',
'l_myelos_percent_trend_from_1095_d',
'l_myelos_percent_trend_from_1825_d',
'l_myelos_percent_trend_from_365_d',
'l_myelos_percent_tulos_norm',
'l_myelos_percent_tulos_norm_val_1095_n',
'l_myelos_percent_tulos_norm_val_1825_n',
'l_myelos_percent_tulos_norm_val_365_n',
'l_neut_percent_trend_from_1095_d',
'l_neut_percent_trend_from_1825_d',
'l_neut_percent_trend_from_365_d',
'l_neut_percent_tulos_norm',
'l_neut_percent_tulos_norm_val_1095_n',
'l_neut_percent_tulos_norm_val_1825_n',
'l_neut_percent_tulos_norm_val_365_n']


In [ ]:
columns_to_read = ['henkilotunnus', 'time_to_dg', 'disease']
time_event = pd.read_csv(data_path + 'lagged_data_' + dis + '3.csv', usecols=columns_to_read, engine='c', low_memory=False)

In [ ]:
time_event

In [ ]:
p_vals = {}

In [ ]:
f_no=0

In [ ]:
n_feat = len(features)

In [ ]:
for feat in features:
    f_no=f_no+1

    print(f'\nFeature {f_no} / {n_feat}')

    try:

        feat_data = pd.read_csv(data_path + 'lagged_data_' + dis + '3.csv', usecols=[feat], engine='c', low_memory=False)

        data = pd.concat([time_event, feat_data], axis=1)
        del feat_data

        if feat == 'sukupuoli_selite':
            data = data[data['sukupuoli_selite'].isin(['Nainen', 'Mies'])]
            data['sukupuoli_selite'] = data['sukupuoli_selite'].replace({'Nainen' : 0, 'Mies' : 1})

        sampled_ids = data['henkilotunnus'].dropna().unique()
        sampled_ids = pd.Series(sampled_ids).sample(n=10000, random_state=42).to_list()
        disease_ids = list(data[data['disease'] == 1]['henkilotunnus'].unique())
        ids = disease_ids + sampled_ids

        data = data[data['henkilotunnus'].isin(ids)]
        data = data.drop(columns=['henkilotunnus'])
    
        # Replace nans with 0
        data.fillna(0, inplace=True)
        
        print(f'{feat}')
    
    
        # Fit the model
        cph = CoxPHFitter() #penalizer=0.1)
        cph.fit(data, duration_col='time_to_dg', event_col='disease')
        
        # Get the p-value for 'feat'
        p_value = cph.summary.loc[feat, 'p']
        print("P-value for ", feat , p_value)
        p_vals[feat] = p_value

    except:
        print(f'Model could not be fitted for {feat}')
    
        # If model could not be fit, set p=1
        p_value = 1
        p_vals[feat] = p_value

    # Save
    with open('univariate_models/' + disease + '_univariate_pvalues.json', 'w') as f:
        json.dump(p_vals, f)

    del data
